In [6]:
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision import datasets
import torchvision.transforms as transforms
from PIL import Image
from typing import List, Dict
import copy

supported_models = ['resnet18', 'resnet50', 'resnet101', 'vgg16_bn', 'vgg19_bn', 'inception_v3']

Model list is available [here](https://github.com/osmr/imgclsmob/blob/master/pytorch/pytorchcv/model_provider.py).

In [5]:
from torchvision.datasets import ImageNet

class_labels = datasets.imagenet.

AttributeError: type object 'ImageNet' has no attribute 'classes'

In [3]:
class ModelFactory():
    _instance = None
    _supported_models = []
    _models = {}
    _class_labels = None

    def __new__(self, supported_models, *args, **kwargs):
        if not self._instance:
            self._instance = super(ModelFactory, self).__new__(self, *args, **kwargs)
            self._supported_models = copy.deepcopy(supported_models)
            self._models = dict()
            self._class_labels = ImageNet.classes
        return self._instance
            
    def get_supported_models(self)->List[str]:
        return self.supported_models

    def get_model(self, model_name):
        m = model_name.lower()
        # import ipdb; ipdb.set_trace()
        try:
            self._supported_models[self._supported_models.index(m)]
        except ValueError as ve:
            raise ValueError(f"Not supported model: {model_name} - {ve.args}")
        model = self._models.get(m)
        if not model: # model is not yet initialized
            model = models.get_model_builder(m)()
            slef._models[m] = model
        return self._models.get(m)

    def get_class_labels(self):
        return self._class_labels

In [4]:
aaa = ModelFactory(supported_models)

AttributeError: type object 'ImageNet' has no attribute 'classes'

In [32]:
aaa.get_model('bbb')

ValueError: Not supported model: bbb - ("'bbb' is not in list",)

In [5]:
aaa.get_supported_models()

['resnet18', 'resnet50', 'resnet101', 'vgg16_bn', 'vgg19_bn', 'inception_v3']

In [6]:
supported_models.append('aaa')

In [7]:
supported_models

['resnet18',
 'resnet50',
 'resnet101',
 'vgg16_bn',
 'vgg19_bn',
 'inception_v3',
 'aaa']

In [8]:
aaa.get_supported_models()

['resnet18', 'resnet50', 'resnet101', 'vgg16_bn', 'vgg19_bn', 'inception_v3']